In [1]:
import json
import os

In [2]:
dataset_path = 'hp/hp1.txt'

In [5]:
os.path.basename(dataset_path).split('.')[0]

'hp1'

In [4]:
def read_list_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, list):
        return data
    else:
        raise ValueError("The JSON file does not contain a list.")

In [7]:
data = read_list_from_json('hp/hp1_questions.json')

In [ ]:
n_list, k_list, q_list = [], [], []
for d in data:
    if d['output'] is None:
        print(f"n = {d['history_upto']}, k = {d['context']}")
        continue
    n_list.append(d['history_upto'])
    k_list.append(d['context'])
    q_list.append(d['output'])
    print(f"number of questions: {len(d['output'])}")

In [21]:
keys = ['question', 'A', 'B', 'C', 'D', 'answer', 'explanation']

In [22]:
for d in data:
    a = [list(q.keys()) == keys for q in d['output']]
    if not all(a):
        print("Fail!")